In [6]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [4]:
spark = SparkSession.builder \
    .appName("CoffeeData") \
    .master("local[*]") \
    .getOrCreate()

spark

In [16]:
df = pd.read_csv(
    r"C:\Users\HP\Desktop\Coffee\data\cleaned\coffee-cleaned.csv",
    sep=';',             
    encoding='utf-8',   
    header =0
)
print(df.shape)
df.head()
print(type(df))

(2188, 8)
<class 'pandas.core.frame.DataFrame'>


In [21]:
target = df["price_per_100g"]
print(target)

0        13.22
1        10.14
2        10.00
3       110.23
4       176.37
         ...  
2183       NaN
2184       NaN
2185       NaN
2186       NaN
2187       NaN
Name: price_per_100g, Length: 2188, dtype: float64


In [18]:
df_train = df[df[target].notna()]
df_missing = df[df[target].isna()]
df_train)

KeyError: "None of [Index([ 13.22,  10.14,   10.0, 110.23, 176.37,   17.2, 705.47, 154.32, 264.55,\n        529.1,\n       ...\n          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,\n          nan],\n      dtype='float64', length=2188)] are in the [columns]"

In [ ]:
X_train = df_train.drop(columns = [target])
y_train = df_train[target]

In [ ]:
X_missing = df_missing.drop(columns = [target])